# Готовим данные

In [4]:
from __future__ import division
from parser import PackToDocks
from udpipe_model import Model
import codecs 
import os
from wiki_ru_wordnet import WikiWordnet
import xlrd
import glob
from sklearn.cluster import KMeans
import artm
import numpy as np
from itertools import combinations
class_list_trump = PackToDocks('trump_labelled.txt', 'data_trump')

class_list_DNR = PackToDocks('lnr_dnr_labelled.txt', 'data_DNR')
    

<b>Расскажу о том как работает метрика качества CPS (название придумал сам) :</b>
    На вход функции принимает два массива - массив "настоящих" кластеров и массив "наших" кластеров (тех, что даст нам kmeans). Далее алгоритм выбирает такую нумерацию кластеров в массиве "наших" кластеров, при которой количество совпадений кластеров между массивами будет наибольшим. Другими словами, подбирается наиболее оптимальная нумерация кластеров. 
    Далее, алгоритм считает количество совпадений кластеров в "настоящем" и "нашем" массиве класте и делит его на длину массива кластеров (количество документов). 
    Метрика принимает значния от 0 до 1, где 0 соответствует нулю совпадений кластеров (в силу особенности алгоритма, ноль никогда не может быть получен), а 1 - идеальной кластеризации. 

<b>Основной недостаток метрики PWS: </b>
Дело в том, что функция считает количество совпадений "наших" кластеров при совпадающих "настоящих" кластерах. Соответственно, чем меньше кластеризированы документы, тем больше метрика покажет значение. Например, при вырожденной кластеризации (в 1 кластер), метрика покажет значение 1.0, что не есть правильно

<b>На мой взгляд, метрика CPS является более адекватной, поэтому будет оценивать качество по ней. Ради интереса посмотрим на соответсвующие значения PWS</b>

In [5]:
def Add(word, dictionary, score):
    if word in dictionary:
        dictionary[word] += score
    else:
        dictionary[word] = score
        
def Distr(dictionary, pos_dic, neg_dic):
    for word in dictionary:
        if dictionary[word] > 0:
            if word in pos_dic:
                pos_dic[word] +=1
            else:
                pos_dic[word] = 1
        elif dictionary[word] < 0:
            if word in neg_dic:
                neg_dic[word] +=1
            else:
                neg_dic[word] = 1
        
def Score(word, wordnet, workbook):
    
    worksheet = workbook.sheet_by_index(0)
    score = 0
    synsets = wordnet.get_synsets(word)
    if bool(synsets):
        synset = synsets[0]
        synlist = [i.lemma() for i in synset.get_words()]
    else:
        synlist = [word]
    #print synlist   
    counter = 0
    for token in synlist:
        flag = False
        for i in range(worksheet.nrows):
            #print 'checking: ' + worksheet.cell(i, 0).value + '==' + token
            if token == worksheet.cell(i, 0).value:
                #print 'WHAT'
                counter +=1
                flag = True
                score += int(worksheet.cell(i, 1).value)
            elif flag:
                break
    if counter:
        score /= counter
    else:
        score = 0
        
    return score

def PWS(class_list, cluster_list):
    PW = 0
    counter = 0
    for i, x in enumerate(class_list):
        for j, y in enumerate(class_list):
            if x == y:
                counter += 1
                PW += int(cluster_list[i] == cluster_list[j])

    return (PW/counter)


def ClearTable(table):
	for i in table:
		if i[0]==i[1]:
			table[i] == False
		else:
			table[i] == 0

def Add(dic, value):
	if value in dic:
		dic[value] +=1
	else:
		dic[value] = 1

def All(dic):
	for i in dic:
		if i[0] == i[1] and dic[i] == False:
			return False
	return True

def GetDict(cluster_list, class_list):
	s = set()
	d = {}
	for x in cluster_list:
		s.add(x)
	for kek in s:
		self_d = {}
		for i in s:
			self_d[i] = 0
		for i in range(len(cluster_list)):
			if cluster_list[i] == kek:
				Add(self_d, class_list[i])
		d[kek] = dict(self_d)
	return d
def Swap(cluster_list, class_list, max_key):
	for i in range(len(cluster_list)):
		if cluster_list[i] == max_key[0]:
			cluster_list[i] = max_key[1]
		elif cluster_list[i] == max_key[1]:
			cluster_list[i] = max_key[0]
def CPS(cluster_list, class_list):
	s = set()
	for x in cluster_list:
		s.add(x)
	table = {}
	for i in combinations(list(s), 2):
		table[i] = 0
	for i in s:
		table[(i, i)] = False
	while True:
		d = GetDict(cluster_list, class_list)
		for line in table:
			if line[0] != line[1]:
				table[line] = d[line[0]][line[1]] + d[line[1]][line[0]]
			else:
				table[line] = d[line[0]][line[1]]
		max_key = None
		for line in table:
			if table[line] > table[(line[0], line[0])] + table[(line[1], line[1])]:
				if max_key == None:
					max_key = line
				elif table[max_key] < table[line]:
					max_key = line
		if max_key == None:
			break
		else:
			Swap(cluster_list, class_list, max_key)

	counter = 0
	for i,x in enumerate(class_list):
		if x == cluster_list[i]:
			counter += 1
	return counter/len(class_list)

In [59]:
model = Model('fi_ud-rus.model')

file_list = [file for file in os.listdir('data_DNR') if os.path.isfile('data_DNR' + '/' + file)]
file_list.sort(key = lambda x: int(x[2:-4]))

with codecs.open('vw.DNR.txt', 'w', 'utf-8') as f:
    for i, file in enumerate(file_list):
        sentences = model.tokenize(codecs.open('data_DNR/' + file, 'r', 'utf-8').read())
        for s in sentences:
            model.tag(s)
            model.parse(s)
        conllu = model.write(sentences, "conllu")

        flag = False
        deps = []
        dep = []
        for line in conllu.split('\n'):
            #print line
            if len(line) > 1 and line[0] == '1':
                flag = True
            elif len(line) == 0:
                flag = False
                if dep != []:
                    deps.append(dep)
                    dep = []
            if flag:
                dep.append(line.split('\t'))

        positive = {}
        negative = {}

        wordnet = WikiWordnet()
        workbook = xlrd.open_workbook('full word_rating_after_coding.xlsx')
        for dep in deps:
            #print dep
            dic = {}
            for line in dep:
                #print u' '.join(line)
                score = Score(line[2], wordnet, workbook)
                #print score
                reverse = []
                if score != 0:
                    if (line[3] == 'NOUN'or 'ADJ' or 'ADV') and len(line) > 5:
                         if dep[int(line[6])-1][3] != 'ADP' and dep[int(line[6])-1][3] != 'PUNCT':
                            Add(dep[int(line[6])-1][2], dic, score)
                    elif line[3] == 'VERB': 
                        for l in dep:
                            if len(l) > 5 and dep[int(l[6])-1] == line and (l[7] == 'nsubj' or 'obj'):
                                Add(l[2], dic, score)

            for l in dep:
                if l[2] == u'не' and len(l) > 5 and dep[int(l[6])-1][2] in dic:
                    dic[dep[int(l[6])-1][2]] *= -1
                    #print type(u'не')          
            #print m
            Distr(dictionary = dic, neg_dic = negative, pos_dic = positive)

        pos = [w + ':' + str(positive[w]) for w in positive]
        neg = [w + ':' + str(negative[w]) for w in negative]
        string = u'{0} |@positive {1} |@negative {2}'.format(str(i), u' '.join(pos), u' '.join(neg))
        f.write(string + '\n')
        #print str(i)+'/'+str(len(file_list)) + ' DONE'

print 'SUCCESS' 


<b>Данные готовы</b>

# Строим тематическую модель

In [6]:
def print_measures(model_artm):
    print 'Sparsity Phi Subj: {} (ARTM)'.format(
        model_artm.score_tracker['SparsityPhiSubjScore'].last_value)

    print 'Sparsity Phi Obj: {} (ARTM)'.format(
        model_artm.score_tracker['SparsityPhiObjScore'].last_value)

    print 'Sparsity Theta: {} (ARTM)'.format(
        model_artm.score_tracker['SparsityThetaScore'].last_value)

    print 'Kernel Subj contrast: {} (ARTM)'.format(
        model_artm.score_tracker['TopicKernelSubjScore'].last_average_contrast)

    print 'Kernel Subj purity: {} (ARTM)'.format(
        model_artm.score_tracker['TopicKernelSubjScore'].last_average_purity)

    print 'Kernel Obj contrast: {} (ARTM)'.format(
        model_artm.score_tracker['TopicKernelObjScore'].last_average_contrast)

    print 'Kernel Obj purity: {} (ARTM)'.format(
        model_artm.score_tracker['TopicKernelObjScore'].last_average_purity)

    print 'Perplexity: {} (ARTM)'.format(
        model_artm.score_tracker['PerplexityScore'].last_value)

    plt.plot(xrange(model_artm.num_phi_updates),
             model_artm.score_tracker['PerplexityScore'].value, 'r--', linewidth=2)
    plt.xlabel('Iterations count')
    plt.ylabel('ARTM perp. (red)')
    plt.grid(True)
    plt.show()
    

<b>Обозначим переменные величины, от которых будет зависить наша модель</b>

In [7]:
N_TOPIC = 5
N_FONT_TOPICS = 2
MIN_TF = 2
NEG_WEIGHT = 0.5
POS_WEIGHT = 1
#[4, 2, 3, 1, 1]

SPARSE_PHI_MAIN = 0
SPARSE_THETA_MAIN = -0.75
SPARSE_PHI_FONT = 1
SPARSE_THETA_FONT = 1
DECORR_PHI = 0
#[-1, -1, 1, 1, 0]

class_ids_phi_main_sparse = ['@negative', '@positive']
class_ids_phi_font_sparse = ['@negative', '@positive']
#['@negative', '@positive']
#['@positive']
#['@negative']
NUM_COLLECTION_PASSES = 15

N_CLUSTERS_TRUMP = 3

N_CLUSTERS_DNR = 10

<b>Создаем пакеты и словарь</b>

In [8]:
batch_vectorizer_trump = None
if len(glob.glob(os.path.join('TM/my_batches_trump', '*.batch'))) < 1:
    batch_vectorizer_trump = artm.BatchVectorizer(data_path='TM/vw.trump.txt', data_format='vowpal_wabbit', target_folder='TM/my_batches_trump')
else:
    batch_vectorizer_trump = artm.BatchVectorizer(data_path='TM/my_batches_trump', data_format='batches')
    
#######################################

batch_vectorizer_DNR = None
if len(glob.glob(os.path.join('TM/my_batches_DNR', '*.batch'))) < 1:
    batch_vectorizer_DNR = artm.BatchVectorizer(data_path='TM/vw.DNR.txt', data_format='vowpal_wabbit', target_folder='TM/my_batches_DNR')
else:
    batch_vectorizer_DNR = artm.BatchVectorizer(data_path='TM/my_batches_DNR', data_format='batches')


In [9]:
dictionary_trump = artm.Dictionary()
#dictionary_trump.gather(data_path='TM/my_batches_trump')
#dictionary_trump.save(dictionary_path='TM/my_batches_trump/my_dictionary_trump')
dictionary_trump.load(dictionary_path='TM/my_batches_trump/my_dictionary_trump.dict')
dictionary_trump.filter(min_tf = MIN_TF)

###################################

dictionary_DNR = artm.Dictionary()
#dictionary_DNR.gather(data_path='TM/my_batches_DNR')
#dictionary_DNR.save(dictionary_path='TM/my_batches_DNR/my_dictionary_DNR')
dictionary_DNR.load(dictionary_path='TM/my_batches_DNR/my_dictionary_DNR.dict')
dictionary_DNR.filter(min_tf = MIN_TF)

artm.Dictionary(name=587df534-63a0-4202-b3e9-065301b7a406, num_entries=524)

<b>Инициализируем модель и добавляем регуляризаторы</b>

In [10]:
topic_names = ['topic_{}'.format(i) for i in xrange(N_TOPIC)]

model_trump = artm.ARTM(num_topics=N_TOPIC, dictionary=dictionary_trump, cache_theta = True, class_ids={'@positive': POS_WEIGHT, '@negative': NEG_WEIGHT})

model_trump.scores.add(artm.PerplexityScore(name = 'PerplexityScore'))
model_trump.scores.add(artm.SparsityPhiScore(name='SparsityPhiNegScore', class_id = '@negative'))
model_trump.scores.add(artm.SparsityPhiScore(name='SparsityPhiPosScore', class_id = '@positive'))
model_trump.scores.add(artm.SparsityThetaScore(name='SparsityThetaScore'))
model_trump.scores.add(artm.TopicKernelScore(name='TopicKernelPosScore', class_id = '@positive'))
model_trump.scores.add(artm.TopicKernelScore(name='TopicKernelNegScore', class_id = '@negative'))

##############################################

topic_names = ['topic_{}'.format(i) for i in xrange(N_TOPIC)]

model_DNR = artm.ARTM(num_topics=N_TOPIC, dictionary=dictionary_DNR, cache_theta = True, class_ids={'@positive': POS_WEIGHT, '@negative': NEG_WEIGHT})

model_DNR.scores.add(artm.PerplexityScore(name = 'PerplexityScore'))
model_DNR.scores.add(artm.SparsityPhiScore(name='SparsityPhiNegScore', class_id = '@negative'))
model_DNR.scores.add(artm.SparsityPhiScore(name='SparsityPhiPosScore', class_id = '@positive'))
model_DNR.scores.add(artm.SparsityThetaScore(name='SparsityThetaScore'))
model_DNR.scores.add(artm.TopicKernelScore(name='TopicKernelPosScore', class_id = '@positive'))
model_DNR.scores.add(artm.TopicKernelScore(name='TopicKernelNegScore', class_id = '@negative'))

In [11]:
model_trump.regularizers.add(artm.SmoothSparsePhiRegularizer(name='sparse_phi_main_regularizer', 
                                                             topic_names = ['topic_{}'.format(i) for i in xrange(N_TOPIC - N_FONT_TOPICS)], 
                                                             class_ids = class_ids_phi_main_sparse))
model_trump.regularizers.add(artm.DecorrelatorPhiRegularizer(name='decorrelator_phi_regularizer'))
model_trump.regularizers.add(artm.SmoothSparsePhiRegularizer(name='sparse_phi_font_regularizer', 
                                                             topic_names = ['topic_{}'.format(i) for i in xrange(N_TOPIC - N_FONT_TOPICS, N_TOPIC)], 
                                                             class_ids = class_ids_phi_font_sparse))

model_trump.regularizers.add(artm.SmoothSparseThetaRegularizer(name='sparse_theta_font_regularizer', 
                                                               topic_names = ['topic_{}'.format(i) for i in xrange(N_TOPIC - N_FONT_TOPICS, N_TOPIC)]))
model_trump.regularizers.add(artm.SmoothSparseThetaRegularizer(name='sparse_theta_main_regularizer', 
                                                               topic_names = ['topic_{}'.format(i) for i in xrange(N_TOPIC - N_FONT_TOPICS)]))

##################################

model_DNR.regularizers.add(artm.SmoothSparsePhiRegularizer(name='sparse_phi_main_regularizer', 
                                                           topic_names = ['topic_{}'.format(i) for i in xrange(N_TOPIC - N_FONT_TOPICS)], 
                                                           class_ids = class_ids_phi_main_sparse))
model_DNR.regularizers.add(artm.DecorrelatorPhiRegularizer(name='decorrelator_phi_regularizer'))
model_DNR.regularizers.add(artm.SmoothSparsePhiRegularizer(name='sparse_phi_font_regularizer', 
                                                           topic_names = ['topic_{}'.format(i) for i in xrange(N_TOPIC - N_FONT_TOPICS, N_TOPIC)], 
                                                           class_ids = class_ids_phi_font_sparse))

model_DNR.regularizers.add(artm.SmoothSparseThetaRegularizer(name='sparse_theta_font_regularizer', 
                                                             topic_names = ['topic_{}'.format(i) for i in xrange(N_TOPIC - N_FONT_TOPICS, N_TOPIC)]))
model_DNR.regularizers.add(artm.SmoothSparseThetaRegularizer(name='sparse_theta_main_regularizer', 
                                                             topic_names = ['topic_{}'.format(i) for i in xrange(N_TOPIC - N_FONT_TOPICS)]))

In [12]:
model_trump.regularizers['sparse_phi_main_regularizer'].tau = SPARSE_PHI_MAIN
model_trump.regularizers['sparse_theta_main_regularizer'].tau = SPARSE_THETA_MAIN
model_trump.regularizers['sparse_phi_font_regularizer'].tau = SPARSE_PHI_FONT
model_trump.regularizers['sparse_theta_font_regularizer'].tau = SPARSE_THETA_FONT
model_trump.regularizers['decorrelator_phi_regularizer'].tau = DECORR_PHI

############################

model_DNR.regularizers['sparse_phi_main_regularizer'].tau = SPARSE_PHI_MAIN
model_DNR.regularizers['sparse_theta_main_regularizer'].tau = SPARSE_THETA_MAIN
model_DNR.regularizers['sparse_phi_font_regularizer'].tau = SPARSE_PHI_FONT
model_DNR.regularizers['sparse_theta_font_regularizer'].tau = SPARSE_THETA_FONT
model_DNR.regularizers['decorrelator_phi_regularizer'].tau = DECORR_PHI


In [13]:
model_trump.fit_offline(batch_vectorizer=batch_vectorizer_trump, num_collection_passes=NUM_COLLECTION_PASSES)

##########################

model_DNR.fit_offline(batch_vectorizer=batch_vectorizer_DNR, num_collection_passes=NUM_COLLECTION_PASSES)

<b>Извлекаем вектора-признаки наших документов. Для этого мы берем только предметные темы, которые мы разреживали</b>

In [14]:
theta_trump = model_trump.get_theta()
lines_trump = np.array([np.array(theta_trump[i])[:N_TOPIC - N_FONT_TOPICS] for i in list(theta_trump)])

###########################

theta_DNR = model_DNR.get_theta()
lines_DNR = np.array([np.array(theta_DNR[i])[:N_TOPIC - N_FONT_TOPICS] for i in list(theta_DNR)])

<b>Ради любопытства можно посмтреть на наши матрицы</b>

In [15]:
print theta_trump
print '*******************'
print theta_DNR

              0         1         2         3         4         5         6    \
topic_0  0.000000  0.000000  0.000000  0.000000  0.000000  0.459943  0.000000   
topic_1  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
topic_2  0.412535  0.000000  0.000000  0.000000  0.000000  0.313959  0.000000   
topic_3  0.220463  0.554869  0.554869  0.656478  0.656478  0.123260  0.296181   
topic_4  0.367002  0.445131  0.445131  0.343522  0.343522  0.102838  0.703819   

              7         8         9    ...       211       212       213  \
topic_0  0.000000  0.175258  0.533812  ...  0.000000  0.000000  0.000000   
topic_1  0.000000  0.000000  0.000000  ...  0.000000  0.000000  0.000000   
topic_2  0.434713  0.000000  0.000000  ...  0.000000  0.000000  0.000000   
topic_3  0.172323  0.191387  0.273895  ...  0.701945  0.111664  0.181682   
topic_4  0.392964  0.633355  0.192294  ...  0.298055  0.888336  0.818318   

              214       215       216       217       21

<b>Дальше небольшой отчет по основным счетчикам</b>

In [16]:
print 'TRUMP: '
print model_trump.score_tracker['PerplexityScore'].last_value
print model_trump.score_tracker['SparsityPhiNegScore'].last_value
print model_trump.score_tracker['SparsityPhiPosScore'].last_value
print model_trump.score_tracker['SparsityThetaScore'].last_value
#print model_trump.score_tracker['TopicKernelPosScore'].last_value
#print model_trump.score_tracker['TopicKernelNegScore'].last_value
print '-----------------------------'
#########################################
print 'DNR: '
print model_DNR.score_tracker['PerplexityScore'].last_value
print model_DNR.score_tracker['SparsityPhiNegScore'].last_value
print model_DNR.score_tracker['SparsityPhiPosScore'].last_value
print model_DNR.score_tracker['SparsityThetaScore'].last_value
#print model.score_tracker['TopicKernelPosScore'].last_value
#print model.score_tracker['TopicKernelNegScore'].last_value

TRUMP: 
6383.78173828
0.531958758831
0.530964493752
0.558371067047
-----------------------------
DNR: 
8840.29199219
0.396987944841
0.416666656733
0.547474741936


<b>Пользуемся методом к-средних, представленным в библиотеке sklearn</b>

In [17]:
kmeans = KMeans(n_clusters = N_CLUSTERS_TRUMP)
kmeans.fit(lines_trump)

cluster_list_trump = kmeans.labels_

#################################

kmeans = KMeans(n_clusters = N_CLUSTERS_DNR)
kmeans.fit(lines_DNR)

cluster_list_DNR = kmeans.labels_

# Оценка качества модели

In [18]:
print 'TRUMP PWS: ' + str(PWS(class_list = class_list_trump, cluster_list = cluster_list_trump))
print 'DNR PWS: ' + str(PWS(class_list = class_list_DNR, cluster_list = cluster_list_DNR))
print '========================================='
print 'TRUMP CPS: ' + str(CPS(class_list = class_list_trump, cluster_list = cluster_list_trump))
print 'DNR CPS: ' + str(CPS(class_list = class_list_DNR, cluster_list = cluster_list_DNR))

TRUMP PWS: 0.777996177996
DNR PWS: 0.571906354515
TRUMP CPS: 0.466063348416
DNR CPS: 0.343434343434


<b>Мы можем визуально сравнивать качества кластеризации, глядя на то, как совпадают векторы кластеров. Индекс списка соответствует номеру документа, значние по этому индексу - кластеру.</b>

In [20]:
print 'TRUMP: '
print 'REAL: ', class_list_trump
print "******************************"
print 'KMEANS: ', cluster_list_trump
print '-----------------------------------------'
print '-----------------------------------------'
print '-----------------------------------------'
print 'DNR: '
print 'REAL: ', class_list_DNR
print "******************************"
print 'KMEANS: ', cluster_list_DNR


TRUMP: 
REAL:  [0, 0, 0, 0, 0, 2, 2, 0, 0, 1, 1, 0, 2, 1, 1, 0, 2, 1, 1, 0, 0, 1, 2, 1, 0, 1, 1, 0, 2, 2, 0, 0, 2, 0, 0, 0, 0, 2, 0, 0, 0, 1, 1, 1, 2, 1, 1, 0, 0, 0, 2, 2, 0, 1, 0, 2, 1, 0, 2, 1, 2, 0, 0, 0, 0, 0, 0, 0, 2, 2, 2, 0, 2, 2, 1, 1, 2, 2, 1, 0, 2, 2, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 1, 1, 1, 2, 2, 2, 1, 2, 2, 2, 1, 2, 1, 2, 2, 1, 2, 2, 2, 2, 2, 1, 1, 2, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 2, 2, 1, 1, 2, 1, 2, 1, 1, 2, 1, 2, 2, 2, 1, 1, 2, 2, 2, 1, 2, 2, 2, 2, 1, 1, 2, 1, 2, 2, 2, 2, 2, 2, 1, 1, 1, 2, 1, 2, 1, 1, 1, 2, 2, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 2, 1, 1, 2, 1, 2, 1, 1, 1, 1, 1, 2, 1, 1, 2, 1, 2, 1, 2, 2, 2, 2, 1, 2, 1, 1, 1, 2, 1, 1, 2, 1, 2, 1, 1, 1, 2, 2, 1, 1]
******************************
KMEANS:  [0 1 1 1 1 2 1 0 1 2 0 0 0 0 1 0 1 1 1 1 1 0 1 1 1 1 0 1 1 1 1 1 2 1 1 0 0
 1 1 1 1 0 1 1 2 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 1 1 1 1 1 2 1 2 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 1 0 1 1 1 1 1 1 1 1 2